1. Ví dụ, trong tháng 1 có 250 interaction, tháng 2 có 300 interaction, tháng 3 có 280, tháng 4 có 320, tháng 5 có 190. tôi lấy 4 tháng đầu để predict tháng 5. Vậy trong trainig_step của tôi nó tính loss như thế nào


Đây là ví dụ rất cụ thể và hay để hình dung luồng đi của dữ liệu. Hãy cùng mổ xẻ quá trình này theo từng bước trong code:

Tình huống: Bạn đang train cho Tháng 5 (Target). Dữ liệu đầu vào (History): Tháng 1, 2, 3, 4. Dữ liệu nhãn (Ground Truth): 190 interaction của Tháng 5.

Quá trình tính Loss trong training_step diễn ra như sau:

Bước 1: Sampler bốc Batch (Dữ liệu thật)
Sampler sẽ lấy toàn bộ (hoặc một phần) dữ liệu của Tháng 5.

Batch chứa: 190 cặp (user, item) ĐÃ XẢY RA trong tháng 5.
current_t (time_idx) = chỉ số của tháng 5.
Bước 2: Model "Hồi tưởng" (Hàm forward)
Hàm training_step gọi self(current_t=Tháng 5). Trong forward, model TUYỆT ĐỐI KHÔNG ĐƯỢC NHÌN đồ thị tháng 5. Nó chỉ làm việc sau:

Lấy đồ thị Tháng 1 -> đưa vào T-GCN -> ra trạng thái $H_1$.
Lấy $H_1$ + đồ thị Tháng 2 -> đưa vào T-GCN -> ra trạng thái $H_2$.
Lấy $H_2$ + đồ thị Tháng 3 -> đưa vào T-GCN -> ra trạng thái $H_3$.
Lấy $H_3$ + đồ thị Tháng 4 -> đưa vào T-GCN -> ra trạng thái $H_{pred}$.
Kết quả: $H_{pred}$ là một bảng Embedding khổng lồ (cho tất cả User/Item) mang ý nghĩa: "Dựa trên những gì đã xảy ra suốt 4 tháng qua, tôi đoán sở thích của mọi người vào đầu tháng 5 là thế này..."

Bước 3: Tính điểm (Scoring)
Quay lại training_step, model cầm bảng $H_{pred}$ để đối chiếu với 190 interaction thật của tháng 5:

Positive Score (Điểm cho cặp thật):
Với interaction thứ 1 (ví dụ: User A thích Book B): Model tra vào bảng $H_{pred}$, lấy vector User A và Book B nhân với nhau.
Làm tương tự cho hết 190 cặp.
=> Ra được 190 con số (hy vọng là số lớn, vì model đoán đúng sở thích).
Negative Score (Điểm cho cặp ngẫu nhiên):
Model bốc ngẫu nhiên 190 cuốn sách khác (User A với Book C "trên trời rơi xuống").
Model tra bảng $H_{pred}$, nhân vector User A với Book C.
=> Ra được 190 con số (hy vọng là số nhỏ, vì User A không thích Book C).
Bước 4: Tính Loss (Trừng phạt)
Hàm BPR Loss sẽ xem xét sự chênh lệch: $$Loss = - \log(\text{sigmoid}(\text{Điểm Thật} - \text{Điểm Ngẫu nhiên}))$$

Nếu Điểm Thật (Tháng 5) >> Điểm Ngẫu nhiên: Loss nhỏ (Model: "Tôi đã đoán đúng xu hướng tháng 5 dựa trên tháng 1-4").
Nếu Điểm Thật < Điểm Ngẫu nhiên: Loss lớn (Model: "Tôi đoán sai bét, cần cập nhật lại trọng số").
Tóm lại
Trong training_step này:

Đầu vào tính toán: Các cạnh nối của tháng 1, 2, 3, 4.
Đáp án để chấm điểm: 190 cạnh nối của tháng 5.
Logic: Dùng quá khứ (1-4) để tạo embedding sao cho nó khớp nhất với tương lai (5).